In [1]:
import pandas as pd
import numpy as np
import os
import regex as re
import spacy as sy
import string
from urllib.parse import urlparse
from nltk.tokenize import TweetTokenizer

nlp_en = sy.load('en_core_web_sm')
all_stopwords = nlp_en.Defaults.stop_words

In [2]:
filename = '/Users/arjunkhanchandani/Desktop/semeval2023task3bundle-v4/data/en/train-labels-subtask-2.txt'
train_sub2_df = pd.read_csv(filename, header=None, sep='\t', names=['id', 'genre'])
print(train_sub2_df.head())

          id                                              genre
0  833042063  Political,Morality,Fairness_and_equality,Exter...
1  832959523  Morality,Security_and_defense,Policy_prescript...
2  833039623  Political,Crime_and_punishment,External_regula...
3  833032367  Political,Crime_and_punishment,Fairness_and_eq...
4  814777937  Political,Morality,Fairness_and_equality,Exter...


In [3]:
dir_name = '/Users/arjunkhanchandani/Desktop/semeval2023task3bundle-v4/data/en/train-articles-subtask-2'

article_df = pd.DataFrame()

numbers = list()
headlines = list()
articles = list()

for file in os.walk(dir_name):
    for filename in file[2]:
        
        number = re.findall('[0-9]+', filename)
        numbers.append(number[0])
        x = dir_name + '/' + filename
        # print(number)
        article = ''
        with open(x) as f:
            lines = f.readlines()
            for i in range(len(lines)):
                if i==0:
                    headline = lines[0]
                    headlines.append(headline)
                elif lines[i]=="\n":
                    continue
                else:
                    # print()
                    article = article + '' + lines[i]
        articles.append(article)
            
article_df['id'] = numbers
article_df['headlines'] = headlines
article_df['articles'] = articles

article_df = article_df.astype({'id': 'int32'})

In [4]:
train_sub2_df

,id,genre
0,833042063,"Political,Morality,Fairness_and_equality,Exter..."
1,832959523,"Morality,Security_and_defense,Policy_prescript..."
2,833039623,"Political,Crime_and_punishment,External_regula..."
3,833032367,"Political,Crime_and_punishment,Fairness_and_eq..."
4,814777937,"Political,Morality,Fairness_and_equality,Exter..."
...,...,...
428,832908978,"Political,Crime_and_punishment,External_regula..."
429,832910505,"Political,Morality,Crime_and_punishment,Qualit..."
430,832917532,"Crime_and_punishment,Political,Legality_Consti..."
431,832913653,"Political,Crime_and_punishment,Fairness_and_eq..."


In [5]:
train_df = train_sub2_df.set_index('id').join(article_df.set_index('id'))
print(train_df.isnull().sum())

train_df

genre        0
headlines    0
articles     0
dtype: int64


,genre,headlines,articles
id,,,
833042063,"Political,Morality,Fairness_and_equality,Exter...",Chelsea Handler Admits She’s ‘Very Sexually At...,Far-left comedienne Chelsea Handler has admitt...
832959523,"Morality,Security_and_defense,Policy_prescript...",How Theresa May Botched\n,Those were the times…\nThe Times page 1 is of ...
833039623,"Political,Crime_and_punishment,External_regula...",Robert Mueller III Rests His Case—Dems NEVER W...,Carload of crazies headed to the White House w...
833032367,"Political,Crime_and_punishment,Fairness_and_eq...",Robert Mueller Not Recommending Any More Indic...,"But of course, this makes no difference to the..."
814777937,"Political,Morality,Fairness_and_equality,Exter...",The Far Right Is Trying to Co-opt the Yellow V...,"This weekend in Manchester, England, Yellow Ve..."
...,...,...,...
832908978,"Political,Crime_and_punishment,External_regula...",Trump Kept Asking Friends If the Mueller Repor...,"On Wednesday, Donald Trump was flying to a mil..."
832910505,"Political,Morality,Crime_and_punishment,Qualit...","The Mueller Circus is Over, Long Live the Muel...",The Mueller circus cost the taxpayers millions...
832917532,"Crime_and_punishment,Political,Legality_Consti...",Robert Mueller Not Recommending Any More Indic...,Special counsel Robert Mueller will not recomm...


In [6]:
print(train_df.genre.value_counts())

Health_and_safety,Quality_of_life                                                                                                                11
Political,Crime_and_punishment,Morality,Legality_Constitutionality_and_jurisprudence                                                             10
Political,Crime_and_punishment,Legality_Constitutionality_and_jurisprudence                                                                       9
Crime_and_punishment,Security_and_defense,Legality_Constitutionality_and_jurisprudence                                                            8
Crime_and_punishment,Morality                                                                                                                     8
                                                                                                                                                 ..
Crime_and_punishment,Morality,Fairness_and_equality,Security_and_defense,Political,Legality_Constitutionality_an

In [7]:
def preprocessing(x, y, df):
    
    pos_tags_final_text = list()
    er_final_text = list()
    preprocessed_text = list()

    for x in df.loc[:,y]:

        tokenizer = TweetTokenizer()
        #tokenizing
        doc = tokenizer.tokenize(x)
        
        # removing links
        tokens = [token for token in doc if not urlparse(token).scheme]
        x = ' '.join(tokens)
        doc = nlp_en(x)
        
        # removing punctuation and white space
        tokens = [token.orth_ for token in doc if not token.is_punct | token.is_space]    
        x = ' '.join(tokens)
        
        # lower case
        x = x.lower()
        doc = nlp_en(x)

        # lemmatization
        tokens = [word.lemma_ for word in doc]   
        x = ' '.join(tokens)
        doc = nlp_en(x)  
        
        # removing punctuation and white space
        tokens = [token.orth_ for token in doc if not token.is_punct | token.is_space]    
        x = ' '.join(tokens)
        doc = nlp_en(x)
        
        # removing individual letters
        tokens = [word.text for word in doc if len(word)>=2]
        x = ' '.join(tokens)  
        # print(x)
        doc = nlp_en(x)
        
        # removing stop words
        tokens = [word for word in doc if not word in all_stopwords]
        list_of_strings  = [i.text for i in tokens]
        x = ' '.join(list_of_strings)
        doc = nlp_en(x)
        
        # Part of speech tagging
        pos_tags = [(i, i.tag_) for i in doc]
        pos_tags_final_text.append(pos_tags)
        
        # entity recognition tagging
        er =  [(i, i.label_, i.label) for i in doc.ents] 
        er_final_text.append(er)
        
        preprocessed_text.append(x)
        
    return pos_tags_final_text, er_final_text, preprocessed_text

In [8]:
pos_tags_final_headlines, er_final_headlines, preprocessed_headlines = preprocessing(headline, 'headlines', train_df)

train_df['preprocessed_headlines'] = preprocessed_headlines
train_df['pos_tags_headlines'] = pos_tags_final_headlines
train_df['er_tags_headlines'] = er_final_headlines

In [9]:
pos_tags_final_articles, er_final_articles, preprocessed_articles = preprocessing(article, 'articles', train_df)

train_df['preprocessed_articles'] = preprocessed_articles
train_df['pos_tags_articles'] = pos_tags_final_articles
train_df['er_tags_articles'] = er_final_articles

In [10]:
train_df.head()

,genre,headlines,articles,preprocessed_headlines,pos_tags_headlines,er_tags_headlines,preprocessed_articles,pos_tags_articles,er_tags_articles
id,,,,,,,,,
833042063,"Political,Morality,Fairness_and_equality,Exter...",Chelsea Handler Admits She’s ‘Very Sexually At...,Far-left comedienne Chelsea Handler has admitt...,chelsea handler admit she very sexually attrac...,"[(chelsea, NNP), (handler, NNP), (admit, VBP),...","[((chelsea, handler), PERSON, 380), ((robert, ...",far leave comedienne chelsea handler have admi...,"[(far, RB), (leave, VB), (comedienne, NNP), (c...","[((chelsea, handler), PERSON, 380), ((fbi), OR..."
832959523,"Morality,Security_and_defense,Policy_prescript...",How Theresa May Botched\n,Those were the times…\nThe Times page 1 is of ...,how theresa may botched,"[(how, WRB), (theresa, NN), (may, MD), (botche...",[],those be the time the time page be of january ...,"[(those, DT), (be, VBP), (the, DT), (time, NN)...","[((january, 18, 2017), DATE, 391), ((the, unit..."
833039623,"Political,Crime_and_punishment,External_regula...",Robert Mueller III Rests His Case—Dems NEVER W...,Carload of crazies headed to the White House w...,robert mueller iii rest his case dem never will,"[(robert, NNP), (mueller, NNP), (iii, NNP), (r...","[((robert, mueller, iii), PERSON, 380), ((dem)...",carload of crazy head to the white house want ...,"[(carload, NN), (of, IN), (crazy, JJ), (head, ...","[((the, white, house), ORG, 383), ((america), ..."
833032367,"Political,Crime_and_punishment,Fairness_and_eq...",Robert Mueller Not Recommending Any More Indic...,"But of course, this makes no difference to the...",robert mueller not recommend any more indictment,"[(robert, NNP), (mueller, NNP), (not, RB), (re...","[((robert, mueller), PERSON, 380)]",but of course this make no difference to the p...,"[(but, CC), (of, IN), (course, NN), (this, DT)...","[((the, new, york, times), ORG, 383), ((late, ..."
814777937,"Political,Morality,Fairness_and_equality,Exter...",The Far Right Is Trying to Co-opt the Yellow V...,"This weekend in Manchester, England, Yellow Ve...",the far right be try to co opt the yellow vest,"[(the, DT), (far, RB), (right, NN), (be, VB), ...",[],this weekend in manchester england yellow vest...,"[(this, DT), (weekend, NN), (in, IN), (manches...","[((this, weekend), DATE, 391), ((manchester, e..."


In [11]:
# converting tweets_df into a csv file
filename = '/Users/arjunkhanchandani/Desktop/semeval2023task3bundle-v4/train_subtask_2.csv'
train_df.to_csv(filename, index=True)